# Estimación de parámetros
## Movimiento Browninano Geométrico

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy import stats
from statsmodels.stats.diagnostic import lilliefors
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

## Lectura de datos
La base de datos fue obtenida de Bloomberg, tiene 200 observaciones en el tiempo.

In [2]:
X = pd.read_csv('datos.csv')['MU US Equity']
X = X.reindex(index=X.index[::-1])
X = np.array(X)
N = len(X)
R = np.zeros(N)
# Retornos instantáneos
for j in range(1,N):
        R[j-1] = (X[j] - X[j-1])/X[j-1]

## Tests de normalidad
Aplicaremos los test de Jarque-Bera, Shappiro Wilks y Lilliefors a los retornos de la base de datos

In [3]:
jb = stats.jarque_bera(R)
lilie = lilliefors(R)
shapiro = stats.shapiro(R)

In [4]:
df = pd.DataFrame()
df['Jarque Bera'] = [1 if jb[1]<0.07 else 0]
df['Shapiro Wilks'] = [1 if shapiro[1]<0.07 else 0]
df['Lilliefors'] = [1 if lilie[1]<0.05 else 0]
df

,Jarque Bera,Shapiro Wilks,Lilliefors
0,1,1,1


## Dimensión fractal

In [5]:
def hFD(a, k_max): #Higuchi FD

    L = []
    x = []
    N = len(a)

    for k in range(1,k_max):
        Lk = 0
        for m in range(0,k):
            #we pregenerate all idxs
            idxs = np.arange(1,int(np.floor((N-m)/k)),dtype=np.int32)
            Lmk = np.sum(np.abs(a[m+idxs*k] - a[m+k*(idxs-1)]))
            Lmk = (Lmk*(N - 1)/(((N - m)/ k)* k)) / k
            Lk += Lmk

        L.append(np.log(Lk/(m+1)))
        x.append([np.log(1.0/ k), 1])

    (p, r1, r2, s)=np.linalg.lstsq(x, L)
    return p[0]

In [6]:
hFD(X, 8)

1.9912197625910415

In [7]:
def hurst(signal):
    tau = []; lagvec = []

    #  Step through the different lags
    for lag in range(2,20):

    #  produce price difference with lag
        pp = np.subtract(signal[lag:],signal[:-lag])

    #  Write the different lags into a vector
        lagvec.append(lag)

    #  Calculate the variance of the difference vector
        tau.append(np.std(pp))

    #  linear fit to double-log graph (gives power)
    m = np.polyfit(np.log10(lagvec),np.log10(tau),1)

    # calculate hurst
    hurst = m[0]

    return hurst

In [8]:
hurst(X)

0.01616094095293045

## Estimación de parámetros
Ajustaremos nuestros datos a un Movimiento Browniano Geométrico:

$$X_t = e^{\mu t + \sigma B_t}$$


In [9]:
dt = 1/len(R)
media = np.mean(R)
varianza = np.var(R)
mu = media/dt
sigma = varianza/dt
print('mu: '+str(mu)+'\n'+'sigma: '+str(sigma))

mu: 1434.3755561309044
sigma: 2052337.9824080004
